In [1]:
import transformers
from transformers import CLIPConfig, CLIPModel, CLIPProcessor, CLIPImageProcessor, CLIPTokenizerFast
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim.lr_scheduler as lr_scheduler
import numpy as np
import random
import math
import scipy.io as sio
import nibabel as nib
from pathlib import Path
from gensim.models import Word2Vec
import re

In [55]:
mat_contents = sio.loadmat('../doi_10.5061_dryad.gt413__v1/subject_1.mat')
words = mat_contents["words"][0]
list_words = []
run = []
length = 4

scan = []
for x in words:
    #print(x[2][0][0])
    if x[2][0][0] < 9:
        word = x[0][0][0][0]
        while('"' in word):
            word = word.replace('"','')
        if len(scan) < length:
            scan.append(word)
        else:
            run.append(" ".join(scan))
            scan = []
    else:
        if len(scan) != 0:
            run.append(" ".join(scan))
            scan = []
        list_words.append(run)
        run = []
        scan = []
        

        

In [56]:
print(len(list_words))
print(list_words)

4
[['Harry had never believed', 'would meet a boy', 'hated more than Dudley,', 'that was before he', 'Draco Malfoy. Still, first-year', 'only had Potions with', "Slytherins, so they didn't", 'to put up with', 'much. Or at least,', "didn't until they spotted", 'notice pinned up in', 'Gryffindor common room that', 'them all groan. Flying', 'would be starting on', '-- and Gryffindor and', 'would be learning together.', 'Typical, said Harry darkly.', 'what I always wanted.', 'make a fool of', 'on a broomstick in', 'of Malfoy. + He', 'been looking forward to', 'to fly more than', "else. You don't know", "you'll make a fool", 'yourself, said Ron reasonably.', "I know Malfoy's always", 'on about how good', 'is at Quidditch, but', "bet that's all talk.", 'Malfoy certainly did talk', 'flying a lot. He', 'loudly about first years', 'getting on the House', 'teams and told long,', 'stories that always seemed', 'end with him narrowly', 'Muggles in helicopters. He', 'the only one, though:', 'way Sea

In [57]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

inputs = processor(text=list_words[0], images=image, return_tensors="pt", padding=True)
outputs = model(**inputs)
# text_embeds is in the same space as the given image
text_embeds = outputs['text_embeds']

# pooler_output is just the text embedding 
# This is what we want to use I think 
pooler_output = outputs['text_model_output']['pooler_output']

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [60]:
print(pooler_output)
print(text_embeds)

tensor([[-0.7876, -1.6431,  1.6882,  ...,  0.2024, -0.9249, -0.7751],
        [ 0.2186, -1.0752,  0.0877,  ..., -1.6650,  0.8878, -1.0070],
        [ 0.2151, -0.9554,  1.6817,  ...,  0.4437,  0.3204,  0.3724],
        ...,
        [-0.7533, -1.1989, -0.1625,  ..., -0.7693, -1.1238, -0.2306],
        [-0.5395,  1.6301,  0.5859,  ..., -1.2589, -1.1547, -0.2034],
        [ 0.5473,  0.7523,  0.1416,  ..., -0.4999, -0.5300,  0.9937]],
       grad_fn=<IndexBackward0>)
tensor([[ 0.0181,  0.0116, -0.0054,  ..., -0.0141, -0.0253,  0.0194],
        [ 0.0138, -0.0112,  0.0184,  ...,  0.0176, -0.0231,  0.0341],
        [ 0.0290,  0.0164,  0.0088,  ...,  0.0096, -0.0047, -0.0099],
        ...,
        [ 0.0365, -0.0249,  0.0473,  ...,  0.0166,  0.0396,  0.0049],
        [ 0.0096,  0.0085,  0.0144,  ..., -0.0061,  0.0084, -0.0091],
        [-0.0115, -0.0300,  0.0041,  ...,  0.0088, -0.0150, -0.0295]],
       grad_fn=<DivBackward0>)
